In [2]:
import numpy as np
import pandas as pd
import numpy as np
import torch
import torch_geometric as pyg
from tqdm.auto import *
import sys
sys.path.append('/Users/xuefengli/24fall/DeepAesthetic/SmartGD')

from smartgd.model import Generator, Discriminator
from smartgd.data import GraphDrawingData
from smartgd.datasets import  RomeDataset
from smartgd.metrics import Stress, Crossings
from smartgd.transformations import Compose, Center, NormalizeRotation, RescaleByStress
from smartgd.criteria import RGANCriterion

/Users/xuefengli/24fall/DeepAesthetic/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
GraphDrawingData.set_optional_fields([
    "edge_pair_metaindex",
    # "face",
    # "rng"
])
dataset = RomeDataset(
    index=pd.read_csv("assets/rome_index.txt", header=None)[0],
)

/Users/xuefengli/24fall/DeepAesthetic/.conda/lib/python3.10/site-packages/torch_geometric/data/dataset.py:213: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  if osp.exists(f)

In [40]:
origin_real_map = {}
index_mapping = {}
datalist = list(dataset)
for i, data in enumerate(datalist):
    origin_real_map[data.G.graph["name"]] = i

for i, data in enumerate(datalist):
    index_mapping[i+1] = data.G.graph["name"]
graph_mapping = origin_real_map
print(len(index_mapping))

11531


In [25]:
target_layouts = np.load("SmartGD/assets/layouts/pmds.npy", allow_pickle=True)
print(type(target_layouts))
print(target_layouts.shape if isinstance(target_layouts, np.ndarray) else "Not an array")  
first_item = target_layouts[0]  # Get the first item
print(first_item[0])
print(type(first_item[0]))

<class 'numpy.ndarray'>
(11531,)
[-96.811238  47.73208 ]
<class 'numpy.ndarray'>


In [26]:
import pandas as pd

data = pd.read_csv("human_label/human_feedback.csv")
result = data.loc[data.groupby("graph_id")["user_id"].idxmin()]
result = result.sort_values("graph_id")
# print(result)
result.to_csv("filtered_file.csv", index=False)


In [28]:
# with open("assets/rome_index.txt", "r") as f:
#     index_mapping = {i + 1: line.strip() for i, line in enumerate(f)}
result["real_graph_id"] = result["graph_id"].map(index_mapping)
print(result)

# with open("assets/rome_index.txt", "r") as index_file:
#             index = index_file.read().strip().split("\n")
#         data_list = map(datatype.static_transform, tqdm(self, desc=f"Transform graphs"))
#         data_dict = {data.G.graph["name"]: data for data in data_list}
#         data_list = [data_dict[name] for name in index]
#         self.data, self.slices = self.collate(list(data_list))

       graph_id  user_id chosen_layout  seconds_spent  is_hard  real_graph_id
0             1       10  kamada_kawai         30.432        1    grafo929.16
2             2        1         neato          8.787        0   grafo3546.36
4             3        1  kamada_kawai          8.943        0   grafo6941.63
6             4        1         neato         12.278        0   grafo4027.45
8             5        1  kamada_kawai         16.167        0   grafo6079.40
...         ...      ...           ...            ...      ...            ...
21463     11527        1           fdp          2.625        0   grafo3463.58
21465     11528        1         neato          1.117        0   grafo2424.35
21467     11529        1  kamada_kawai          1.646        0  grafo11007.93
21469     11530        1           fa2          0.940        0   grafo7882.67
21471     11531        1           fdp          1.529        0   grafo5901.35

[11531 rows x 6 columns]


In [29]:
# with open("assets/rome_index.txt", "r") as f:
#     graph_mapping = {line.strip() : i for i, line in enumerate(f)}
print(len(graph_mapping))
print(graph_mapping)
print(len(index_mapping))


11531
{'grafo929.16': 0, 'grafo3546.36': 1, 'grafo6941.63': 2, 'grafo4027.45': 3, 'grafo6079.40': 4, 'grafo5960.55': 5, 'grafo7363.39': 6, 'grafo431.13': 7, 'grafo6971.81': 8, 'grafo5906.34': 9, 'grafo11399.32': 10, 'grafo5880.38': 11, 'grafo3538.40': 12, 'grafo5379.39': 13, 'grafo9962.97': 14, 'grafo9009.71': 15, 'grafo8993.87': 16, 'grafo569.28': 17, 'grafo760.13': 18, 'grafo9250.73': 19, 'grafo7695.56': 20, 'grafo6553.46': 21, 'grafo5587.48': 22, 'grafo4936.83': 23, 'grafo11453.91': 24, 'grafo2415.13': 25, 'grafo7440.42': 26, 'grafo9600.65': 27, 'grafo6943.82': 28, 'grafo1951.14': 29, 'grafo1592.27': 30, 'grafo1239.28': 31, 'grafo6742.77': 32, 'grafo3827.42': 33, 'grafo8352.89': 34, 'grafo5712.36': 35, 'grafo9770.79': 36, 'grafo2005.30': 37, 'grafo9070.67': 38, 'grafo6496.52': 39, 'grafo1708.21': 40, 'grafo10856.91': 41, 'grafo7861.87': 42, 'grafo11078.100': 43, 'grafo3115.59': 44, 'grafo6419.45': 45, 'grafo8533.60': 46, 'grafo3159.38': 47, 'grafo386.15': 48, 'grafo1398.24': 49, 'gr

In [31]:
layout_mapping = (
    result.groupby("real_graph_id")["chosen_layout"]
    .first()  # Get the first chosen layout (assuming it's consistent per graph_id)
    .to_dict()
)
print(layout_mapping)

{'grafo1000.14': 'neato', 'grafo10000.38': 'kamada_kawai', 'grafo10001.32': 'fdp', 'grafo10002.40': 'fdp', 'grafo10003.40': 'kamada_kawai', 'grafo10004.32': 'neato', 'grafo10005.39': 'neato', 'grafo10006.98': 'fdp', 'grafo10007.31': 'neato', 'grafo10008.42': 'fdp', 'grafo10009.31': 'sfdp', 'grafo1001.12': 'fa2', 'grafo10010.39': 'fdp', 'grafo10011.31': 'neato', 'grafo10012.40': 'kamada_kawai', 'grafo10013.31': 'fdp', 'grafo10014.39': 'kamada_kawai', 'grafo10015.39': 'fdp', 'grafo10016.39': 'neato', 'grafo10017.96': 'neato', 'grafo10018.31': 'fdp', 'grafo10019.38': 'kamada_kawai', 'grafo1002.20': 'neato', 'grafo10020.38': 'spring', 'grafo10021.39': 'kamada_kawai', 'grafo10022.31': 'kamada_kawai', 'grafo10023.39': 'kamada_kawai', 'grafo10024.32': 'neato', 'grafo10025.31': 'neato', 'grafo10026.32': 'neato', 'grafo10027.38': 'kamada_kawai', 'grafo10028.96': 'kamada_kawai', 'grafo10029.40': 'kamada_kawai', 'grafo1003.11': 'spring', 'grafo10030.38': 'kamada_kawai', 'grafo10031.38': 'neato', 

In [32]:
from decimal import Decimal
import json
with open("rome_table_data.json", "r") as f:
    json_data = json.load(f, parse_float=Decimal)

In [33]:
results = {}
for item in json_data:
    graph_id = item.get('graph_id')
    
    # Check if this graph ID is in our layout mapping
    if graph_id in layout_mapping:
        method = item.get('method', '')
        requested_layout = layout_mapping[graph_id]
        
        # Check if the method contains the requested layout
        if requested_layout.lower() == method.lower():
            results[graph_id] = {
                'graph_id': graph_id,
                'layout': requested_layout,
                'positions': item.get('__pos__')
            }


In [1]:
for item in json_data:
    graph_id = item.get('graph_id')
    method = item.get('method', '')
    if graph_id == "grafo7312.41":
        print(method)

NameError: name 'json_data' is not defined

In [34]:
class DecimalEncoder(json.JSONEncoder):
    """Custom encoder to handle Decimal objects"""
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return super(DecimalEncoder, self).default(obj)

with open("chosen_layouts.json", 'w') as f:
            json.dump(results, f, cls=DecimalEncoder, indent=2)

In [35]:
# handle not in chosen layout
outlier = []
for graph_id in layout_mapping:
    if graph_id not in results:
        outlier.append(graph_id)

print(len(outlier))
print(outlier)

0
[]


In [ ]:
print(result[result["real_graph_id"] == "grafo7312.41"])

      graph_id  user_id chosen_layout  seconds_spent  is_hard real_graph_id
3136      1641        1         neato          3.138        0  grafo7312.41
grafo7312.41
{}


In [36]:

aligned_positions = []
count = 0
for i in range(0, 11531):
    graph_id = index_mapping[i+1]
    if graph_id not in layout_mapping:
        break
    if graph_id not in results:
        positions = target_layouts[graph_mapping[graph_id]]
        aligned_positions.append(positions)
    else:
        positions = results[graph_id].get("positions")
        float_positions = np.array([[float(coord) for coord in point] for point in positions], dtype=float)
        aligned_positions.append(float_positions)

aligned_positions_array = np.array(aligned_positions, dtype=object)
np.save("aligned_positions.npy", aligned_positions_array)


In [42]:
import numpy as np

chosen_layouts = np.load("aligned_positions.npy", allow_pickle=True)
target_layouts = np.load("assets/layouts/pmds.npy", allow_pickle=True)

# Validate shapes
chosen_shapes = [item.shape for item in chosen_layouts]
target_shapes = [item.shape for item in target_layouts]

# Compare all shapes
for idx, (new, origin) in enumerate(zip(chosen_shapes, target_shapes)):
    if new != origin:
        print(f"Shape mismatch at index {idx}: {new} vs {origin}")


In [41]:
import numpy as np
chosen_layouts = np.load("aligned_positions.npy", allow_pickle=True)
print(type(chosen_layouts))
print(chosen_layouts.shape if isinstance(chosen_layouts, np.ndarray) else "Not an array")  
first_item = chosen_layouts[0]  # Get the first item
chosen_shape = []
for item in chosen_layouts:
    chosen_shape.append(item.shape)
print(chosen_shape)

pmds = []
target_layouts = np.load("assets/layouts/pmds.npy", allow_pickle=True)
print(target_layouts.shape if isinstance(target_layouts, np.ndarray) else "Not an array")  
for item in target_layouts:
    pmds.append(item.shape)
print(pmds)
for new, origin in zip(chosen_shape, pmds):
    if new != origin:
        print("different", new, origin)


<class 'numpy.ndarray'>
(11531,)
[(16, 2), (36, 2), (63, 2), (45, 2), (40, 2), (55, 2), (39, 2), (13, 2), (81, 2), (34, 2), (32, 2), (38, 2), (40, 2), (39, 2), (97, 2), (71, 2), (87, 2), (28, 2), (13, 2), (73, 2), (56, 2), (46, 2), (48, 2), (83, 2), (91, 2), (13, 2), (42, 2), (65, 2), (82, 2), (14, 2), (27, 2), (28, 2), (77, 2), (42, 2), (89, 2), (36, 2), (79, 2), (30, 2), (67, 2), (52, 2), (21, 2), (91, 2), (87, 2), (100, 2), (59, 2), (45, 2), (60, 2), (38, 2), (15, 2), (24, 2), (20, 2), (49, 2), (39, 2), (52, 2), (20, 2), (95, 2), (79, 2), (52, 2), (94, 2), (86, 2), (36, 2), (98, 2), (41, 2), (65, 2), (12, 2), (61, 2), (29, 2), (94, 2), (15, 2), (61, 2), (93, 2), (78, 2), (75, 2), (90, 2), (67, 2), (56, 2), (38, 2), (52, 2), (42, 2), (39, 2), (15, 2), (78, 2), (23, 2), (27, 2), (42, 2), (56, 2), (71, 2), (22, 2), (92, 2), (99, 2), (68, 2), (83, 2), (15, 2), (95, 2), (61, 2), (28, 2), (40, 2), (66, 2), (91, 2), (82, 2), (65, 2), (64, 2), (62, 2), (43, 2), (48, 2), (74, 2), (72, 2), (2